In [1]:
using Revise

In [84]:
using RigidBodyDynamics
using RigidBodyDynamics: Contact
using MeshCat
using MeshCatMechanisms
using Blink: Window, body!
using StaticArrays
using LearningMPC
using Gurobi

In [6]:
vis = Visualizer()
open(vis, Window())

Blink.AtomShell.Window(1, Blink.AtomShell.Electron(Process(`/home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/deps/atom/electron /home/rdeits/locomotion/explorations/learning-mpc-2/packages/v0.6/Blink/src/AtomShell/main.js port 5126`, ProcessRunning), TCPSocket(RawFD(65) active, 0 bytes waiting), Dict{String,Any}(Pair{String,Any}("callback", Blink.#3))), Blink.Page(1, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(58) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 1, Nullable{Any}(true))))

In [79]:
world = RigidBody{Float64}("world")
mechanism = Mechanism(world; gravity=SVector(0, 0, -9.81))

frame = CartesianFrame3D("core")
inertia = SpatialInertia(frame, SDiagonal(1 + 2.5, 1 + 2.5, 0.5), SVector(0., 0, 5.), 10.)
core = RigidBody(inertia)
joint = Joint("floating_base", QuaternionFloating{Float64}())
attach!(mechanism, world, core, joint)
position_bounds(joint) .= RigidBodyDynamics.Bounds(-10, 10)
velocity_bounds(joint) .= RigidBodyDynamics.Bounds(-1000, 1000)
effort_bounds(joint) .= RigidBodyDynamics.Bounds(0, 0)

frame = CartesianFrame3D("r_foot")
inertia = SpatialInertia(frame, SDiagonal(0.1 + .25, 0.1 + .25, 0.025), SVector(0., 0, -0.5), 1.0)
foot = RigidBody(inertia)
joint = Joint("r_hip", Revolute(SVector(0., 1, 0)))
joint_to_parent = Transform3D(frame_before(joint), default_frame(core), SVector(0, -0.2, 0))
attach!(mechanism, core, foot, joint, joint_pose=joint_to_parent)
position_bounds(joint) .= RigidBodyDynamics.Bounds(-10, 10)
velocity_bounds(joint) .= RigidBodyDynamics.Bounds(-1000, 1000)
effort_bounds(joint) .= RigidBodyDynamics.Bounds(-10, 10)

frame = CartesianFrame3D("l_foot")
inertia = SpatialInertia(frame, SDiagonal(0.1 + .25, 0.1 + .25, 0.025), SVector(0., 0, -0.5), 1.0)
foot = RigidBody(inertia)
joint = Joint("l_hip", Revolute(SVector(0., 1, 0)))
joint_to_parent = Transform3D(frame_before(joint), default_frame(core), SVector(0, 0.2, 0))
attach!(mechanism, core, foot, joint, joint_pose=joint_to_parent)
position_bounds(joint) .= RigidBodyDynamics.Bounds(-10, 10)
velocity_bounds(joint) .= RigidBodyDynamics.Bounds(-1000, 1000)
effort_bounds(joint) .= RigidBodyDynamics.Bounds(-10, 10)


floor = HalfSpace3D(Point3D(default_frame(world), 0., 0, 0), FreeVector3D(default_frame(world), 0., 0, 1))
add_environment_primitive!(hopper, floor)
contactmodel = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(0.8, 20e3, 100.))

for foot in [findbody(mechanism, "r_foot"), findbody(mechanism, "l_foot")]
    add_contact_point!(foot, Contact.ContactPoint(Point3D(default_frame(foot), 0., 0, -1), contactmodel))
end

function init!(state::MechanismState)
    m = state.mechanism
    set_configuration!(state, findjoint(m, "floating_base"), [1, 0, 0, 0, 0, 0, 1])
    set_configuration!(state, findjoint(m, "r_hip"), [0])
    set_configuration!(state, findjoint(m, "l_hip"), [0])
    state
end

init! (generic function with 1 method)

In [80]:
delete!(vis)
mvis = MechanismVisualizer(mechanism, Skeleton(randomize_colors=true), vis)

copy!(mvis, init!(MechanismState(mechanism)))

In [85]:
body!(Window(), manipulate!(mvis))

Blink.Page(3, WebSockets.WebSocket{TCPSocket}(TCPSocket(RawFD(60) active, 0 bytes waiting), true, CONNECTED::WebSockets.ReadyState = 1), Dict{String,Any}(Pair{String,Any}("webio", WebIO.#109),Pair{String,Any}("callback", Blink.#3)), Future(1, 1, 3, Nullable{Any}(true)))

ERROR (unhandled task failure): OutOfMemoryError()
Stacktrace:
 [1] setindex!(::ObjectIdDict, ::Any, ::Any) at ./associative.jl:430
 [2] preserve_handle at ./libuv.jl:51 [inlined]
 [3] stream_wait(::Timer, ::Condition, ::Vararg{Condition,N} where N) at ./stream.jl:40
 [4] wait(::Timer) at ./event.jl:357
 [5] (::Base.##302#303{IJulia.#send_stdout,Timer})() at ./event.jl:430


In [81]:
qq = [10, 10, 10, 10, 1, 1, 1, 0.1, 0.1]
qv = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.01, 0.01]
Q = diagm(vcat(qq, qv))
R = diagm(fill(1e-6, num_velocities(mechanism)))
contacts = [
    Point3D(default_frame(findbody(mechanism, name)), 0., 0, -1) for name in ["r_foot", "l_foot"]
]
Δt = 0.01
lqrsol = LQRSolution(init!(MechanismState(mechanism)), Q, R, Δt, contacts)


LoadError: [91mBase.LinAlg.SingularException(6)[39m

In [53]:
floor_obstacle = LCPSim.planar_obstacle(default_frame(world), SVector(0., 0, 1), SVector(0., 0, 0))
env = LCPSim.Environment([
    (body, Point3D(default_frame(body), 0., 0, -1), floor_obstacle)
        for body in findbody.(mechanism, ["r_foot", "l_foot"])
    ]);

In [77]:
state = init!(MechanismState(mechanism))
set_velocity!(state, findjoint(mechanism, "floating_base"), [0, 0.01, 0, 0, 0, 0])
solver = GurobiSolver(Gurobi.Env(); OutputFlag=0)
N = 200
results = LCPSim.simulate(state, x -> zeros(num_velocities(state)), env, Δt, N, solver);
setanimation!(mvis, results)

Academic license - for non-commercial use only


true

In [69]:
configuration(results[20].state)

15-element SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}:
  0.852754 
 -0.242991 
 -0.28263  
  0.365904 
  0.407021 
 -0.0916046
  1.08383  
  0.932243 
 -0.254608 
  0.086926 
 -0.241955 
  0.899992 
 -0.369885 
  0.0607409
 -0.22251  

In [58]:
velocity(results[end].state)

12-element SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}:
  13.542  
 -12.4222 
   3.01264
   5.26784
   6.79893
  -6.79936
  -6.9131 
  28.9165 
 -70.7398 
  -7.68643
  21.6814 
 -65.3274 

In [57]:
length(results)

27